In [17]:
# from langchain_anthropic import ChatAnthropic
# import os

# anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
# llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0.2, max_tokens=1024, anthropic_api_key=anthropic_api_key)

# # claude-3-sonnet-20240229


In [5]:
from IPython.display import display
from IPython.display import Markdown
from langchain_core.prompts import ChatPromptTemplate
import google.generativeai as genai
import textwrap

In [6]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [17]:
# models/embedding-001
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
import google.generativeai as genai
import os
api_key=os.environ.get("GOOGLE_API_KEY")


In [11]:
llm = genai.GenerativeModel(model_name = "gemini-pro")

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [16]:
from langchain_google_genai import GoogleGenerativeAI

In [18]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

**Pros of Python:**

* **Readability and simplicity:** Python's syntax is designed to be clear and concise, making it easy to read and understand, even for beginners.
* **Extensive libraries:** Python has a vast ecosystem of libraries and frameworks, providing pre-built functionality for various tasks, such as data science, machine learning, web development, and more.
* **Flexibility and versatility:** Python is a general-purpose language suitable for developing a wide range of applications, from small scripts to complex enterprise systems.
* **Cross-platform support:** Python can run on multiple operating systems, including Windows, macOS, Linux, and Unix, without requiring platform-specific recompilation.
* **Beginner-friendly:** Python's simplicity and ease of learning make it an excellent choice for beginners who are new to programming.
* **Strong community support:** Python has a large and active community, providing resources, documentation, and support to developers.

**Cons of 

In [19]:
from langchain.prompts import PromptTemplate

In [20]:
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "How much is 2+2?"
print(chain.invoke({"question": question}))

Question: How much is 2+2?

Answer: 4


In [25]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("spark.pdf")

docs = loader.load()

In [40]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en")

In [41]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [42]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [43]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can let you visualize test results.'

In [44]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [46]:
response = retrieval_chain.invoke({"input": "what is PySpark"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

This context does not mention anything about PySpark, so I cannot answer this question from the provided context.
